<a href="https://colab.research.google.com/github/sharat-shankar/10kGNAD/blob/master/J045_NLP_M2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install pyLDAvis

# **Libraries and Loading Data**

In [1]:
# Library Requirements
import os
import re
import nltk
import scipy
import gensim
import pickle 
import pyLDAvis
import itertools
import numpy as np
import pandas as pd
import pyLDAvis.gensim
from gensim import models
from zipfile import ZipFile
from bs4 import BeautifulSoup
from google.colab import files
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from itertools import combinations
from sklearn.pipeline import Pipeline
from string import digits,punctuation 
from nltk.tokenize import word_tokenize
from pyLDAvis import sklearn as sklearn_lda
from gensim.models.ldamodel import LdaModel
from gensim.sklearn_api import LdaTransformer
from sklearn.preprocessing import FunctionTransformer
from gensim.test.utils import get_tmpfile, common_texts
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA
nltk.download('punkt')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords') 
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [174]:
! wget https://github.com/amsurve/J050_sem6_nlp/blob/master/datasets/10k-german-news-articles.zip?raw=true

--2020-03-03 14:47:13--  https://github.com/amsurve/J050_sem6_nlp/blob/master/datasets/10k-german-news-articles.zip?raw=true
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/amsurve/J050_sem6_nlp/raw/master/datasets/10k-german-news-articles.zip [following]
--2020-03-03 14:47:13--  https://github.com/amsurve/J050_sem6_nlp/raw/master/datasets/10k-german-news-articles.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/amsurve/J050_sem6_nlp/master/datasets/10k-german-news-articles.zip [following]
--2020-03-03 14:47:14--  https://media.githubusercontent.com/media/amsurve/J050_sem6_nlp/master/datasets/10k-german-news-articles.zip
Resolving media.githubusercontent.com (media.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.13

In [175]:
# Unzipping the Datasets
! mkdir 10kGNAD
! unzip 10k-german-news-articles.zip?raw=true -d 10kGNAD

mkdir: cannot create directory ‘10kGNAD’: File exists
Archive:  10k-german-news-articles.zip?raw=true
  inflating: 10kGNAD/Annotations.csv  
  inflating: 10kGNAD/Annotations_consolidated.csv  
  inflating: 10kGNAD/Articles.csv    
  inflating: 10kGNAD/Categories.csv  
  inflating: 10kGNAD/CrossValSplit.csv  
  inflating: 10kGNAD/Newspaper_Staff.csv  
  inflating: 10kGNAD/Posts.csv       


In [0]:
# Import datasets into pandas dataframe
Articles = pd.read_csv('10kGNAD/Articles.csv')

In [0]:
Articles = Articles.drop(['ID_Article','Path','publishingDate'],1)
articles = Articles[0:1000]

# **Functions for SKLEARN PIPELINE**

In [0]:
# Function to extract the body from the HTML text
def extract_body_per_row(str):
  soup = BeautifulSoup(str)
  for script in soup(["script", "style"]):
      script.extract()
  clean_text = soup.get_text()
  lines = (line.strip() for line in clean_text.splitlines())
  chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
  final_text = '\n'.join(chunk for chunk in chunks if chunk)
  return(final_text)

def html_processed_df(df):
  return df.apply(extract_body_per_row)

get_html_body = FunctionTransformer(html_processed_df, validate=False)

In [0]:
# Function to extract tokens from the body and remove stopwords, punctuations and digits
remove_digits = str.maketrans('', '', digits)
stop_words = set(stopwords.words('german'))

def token_extractifier(str):
  str = str.lower()
  str = str.translate(remove_digits)
  str = re.sub(r'[^\w\s]',' ',str)
  token = word_tokenize(str)
  words = [word for word in token if word not in stop_words and word not in punctuation]
  return words

def tokenized_df(df):
   df = df.apply(token_extractifier)
   global dictionary
   dictionary = gensim.corpora.Dictionary(df)
   return df

get_tokens = FunctionTransformer(tokenized_df, validate=False)

In [0]:
# Fuction to convert the processed doc into Count Vectorizer dataframe
def dummy_function(doc):
    return doc

def convert_to_count_vec(df):
  global vectorizer
  global vectors
  vectorizer = CountVectorizer(analyzer='word',tokenizer=dummy_function,preprocessor=dummy_function,token_pattern=None)
  vectors = vectorizer.fit_transform(df)
  return vectors

get_count_vec_df = FunctionTransformer(convert_to_count_vec, validate=False)

# **Implementing Pipeline and Fitting the SKLEARN LDA Model**

In [7]:
# LDA Pipeline
number_topics = 5
number_words = 10
# Create LDA model
lda = LDA(n_components=number_topics, n_jobs=-1)
LDA_pipeline = Pipeline(memory=None,steps=[('Extract_Body',get_html_body),('Tokenize',get_tokens),('Corpus_DataFrame',get_count_vec_df),("LDA_model",lda)],verbose=True)

# LDA Model fit on the Body of Articles
lda_model = LDA_pipeline.fit(articles['Body'])

[Pipeline] ...... (step 1 of 4) Processing Extract_Body, total=   1.2s
[Pipeline] .......... (step 2 of 4) Processing Tokenize, total=   2.7s
[Pipeline] .. (step 3 of 4) Processing Corpus_DataFrame, total=   0.3s
[Pipeline] ......... (step 4 of 4) Processing LDA_model, total=   6.6s


In [8]:
# Printing the topics in LDA model for Body of the articles

def print_topics(model, vectorizer, top_words):
    words = vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([words[i] for i in topic.argsort()[:-top_words - 1:-1]]))
        
print_topics(lda_model.named_steps['LDA_model'], vectorizer, number_words)

Topic #0:
mehr standard wurde sei gibt euro schon zwei sagte wurden
Topic #1:
spö fpö mehr eu övp wurde sei schon koalition immer
Topic #2:
prozent wien österreich wurde fpö mehr sei zwei menschen spö
Topic #3:
mehr österreich wurde sei schon immer trainer seit zwei neue
Topic #4:
mehr prozent wurde sei euro zwei schon millionen apa laut


# **Predicting the Topic for each body of the article**

In [0]:
# Getting the TOP Topic Predicted for each row
topics = []
words = vectorizer.get_feature_names()
for topic_idx, topic in enumerate(lda_model.named_steps['LDA_model'].components_):
    topics.append(" ".join([words[i] for i in topic.argsort()[:-number_words - 1:-1]]))
feature_names = vectorizer.get_feature_names()    
probabs = lda_model.named_steps['LDA_model'].transform(pd.DataFrame((vectors.todense()).tolist(),columns = feature_names))
top_topics = []
for probs in probabs:
  top_topics.append(topics[np.argmax(probs)])

# **Cosine Similarity to get the Accuracy of the Model**

In [0]:
# Convert Titles and Predicted Titles to Vetors to Calculating Cosine Similarity
tfidf_vectorizer = TfidfVectorizer(analyzer='word')
actual_title_vectors = tfidf_vectorizer.fit_transform(articles['Title'])
predicted_title_vectors = tfidf_vectorizer.transform(top_topics)

In [0]:
# Converting Vectors to List
actual_title_list = actual_title_vectors.todense().tolist()
predicted_title_list = predicted_title_vectors.todense().tolist()

In [0]:
# Calculating Cosine Similarity between actual and predicted titles
cosine_similarities = []
for i in range(len(top_topics)):
  cosine_similarities.append(scipy.spatial.distance.cosine(actual_title_list[i], predicted_title_list[i]))

# **Accuracy of the Model: Between Actual and Predicted Titles**

In [13]:
# Accuracy of the model by counting the cosine similarity values of 1
count = 0
for i in cosine_similarities:
  if i > 0.9:
    count = count + 1
accuracy = (count/len(cosine_similarities))*100
print("Accuracy of the model: ",accuracy)

Accuracy of the model:  95.0


# **PyLDAvis Implementation**



In [0]:
LDAvis = sklearn_lda.prepare(lda_model.named_steps['LDA_model'], vectors, vectorizer)

In [0]:
# Download the HTML file to view Visualization of Topic Models by pyLDAvis
pyLDAvis.save_html(LDAvis, './J045_ldavis'+ str(number_topics) +'.html')

In [144]:
pyLDAvis.display(LDAvis)